In [1]:
from influxdb.commands import InfluxDBQueries, DataProcessor, LSTMModel
import warnings
import pandas as pd
import tensorflow as tf

tf.config.set_visible_devices([], 'GPU')
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

In [2]:
df_train = InfluxDBQueries.get_data_from_influx(groupby_time='1h', pair='BTCUSDT', start="2022-01-01T01:00:00Z",
                                                stop="2025-01-01T01:00:00Z")
df_val = InfluxDBQueries.get_data_from_influx(groupby_time='1h', pair='BTCUSDT', start="2025-01-01T01:00:00Z",
                                              stop="2025-03-01T01:00:00Z")
df_test = InfluxDBQueries.get_data_from_influx(groupby_time='1h', pair='BTCUSDT', start="2025-03-01T01:00:00Z",
                                               stop="2025-04-20T01:00:00Z")

In [3]:
df_combined = pd.concat([df_train, df_test, df_val])

In [4]:
df_combined, scaler, columns, sclaer_y = DataProcessor.add_features(df_combined, classification=True, group_time='1h')

In [5]:
df_combined.dropna(inplace=True)

In [6]:
columns, boruta = DataProcessor.select_features_boruta(df_combined, columns, classification=True)

Iteration: 	1 / 200
Confirmed: 	0
Tentative: 	42
Rejected: 	0
Iteration: 	2 / 200
Confirmed: 	0
Tentative: 	42
Rejected: 	0
Iteration: 	3 / 200
Confirmed: 	0
Tentative: 	42
Rejected: 	0
Iteration: 	4 / 200
Confirmed: 	0
Tentative: 	42
Rejected: 	0
Iteration: 	5 / 200
Confirmed: 	0
Tentative: 	42
Rejected: 	0
Iteration: 	6 / 200
Confirmed: 	0
Tentative: 	42
Rejected: 	0
Iteration: 	7 / 200
Confirmed: 	0
Tentative: 	42
Rejected: 	0
Iteration: 	8 / 200
Confirmed: 	11
Tentative: 	27
Rejected: 	4
Iteration: 	9 / 200
Confirmed: 	11
Tentative: 	27
Rejected: 	4
Iteration: 	10 / 200
Confirmed: 	11
Tentative: 	27
Rejected: 	4
Iteration: 	11 / 200
Confirmed: 	11
Tentative: 	27
Rejected: 	4
Iteration: 	12 / 200
Confirmed: 	14
Tentative: 	24
Rejected: 	4
Iteration: 	13 / 200
Confirmed: 	14
Tentative: 	24
Rejected: 	4
Iteration: 	14 / 200
Confirmed: 	14
Tentative: 	24
Rejected: 	4
Iteration: 	15 / 200
Confirmed: 	14
Tentative: 	24
Rejected: 	4
Iteration: 	16 / 200
Confirmed: 	15
Tentative: 	21
Rejec

In [7]:
df_train = df_combined.loc[df_combined.index.strftime('%Y-%m-%dT%H:%M:%SZ') < df_val.index[0].strftime('%Y-%m-%dT%H:%M:%SZ')]
df_val = df_combined.loc[
    (df_combined.index.strftime('%Y-%m-%dT%H:%M:%SZ') >= df_val.index[0].strftime('%Y-%m-%dT%H:%M:%SZ')) &
    (df_combined.index.strftime('%Y-%m-%dT%H:%M:%SZ') < df_test.index[0].strftime('%Y-%m-%dT%H:%M:%SZ'))]
df_test = df_combined.loc[
    df_combined.index.strftime('%Y-%m-%dT%H:%M:%SZ') >= df_test.index[0].strftime('%Y-%m-%dT%H:%M:%SZ')]

In [8]:
df_train = df_train.dropna()
df_test = df_test.dropna()
df_val = df_val.dropna()

In [9]:
X_train, y_train = DataProcessor.make_sequences(df_train, columns_to_select=columns, window_size=48,forecast_horizon=1, classification=True)

In [10]:
X_val, y_val = DataProcessor.make_sequences(df_val, columns_to_select=columns, window_size=48, forecast_horizon=1, classification=True)

In [11]:
lstm_forecaster = LSTMModel(input_shape=(48, len(columns)), output_steps=1, classification=True)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 48, 64)         │         3,904 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 48, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 48, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 48, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 48, 128)        │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 48, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 48, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 80)             │        66,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 80)             │           320 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 80)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            81 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 170,769 (667.07 KB)

 Trainable params: 170,225 (664.94 KB)

 Non-trainable params: 544 (2.12 KB)

In [12]:
lstm_forecaster.fit(X_train, y_train, X_val, y_val, epochs=300, batch_size=32, patience=20)

Training started with EarlyStopping (patience=20)...
Epoch 1/300
811/811 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - accuracy: 0.4944 - auc: 0.4974 - loss: 0.8785
Epoch 1: val_loss improved from inf to 0.70480, saving model to models/best_model.keras
811/811 ━━━━━━━━━━━━━━━━━━━━ 65s 77ms/step - accuracy: 0.4944 - auc: 0.4974 - loss: 0.8784 - val_accuracy: 0.5110 - val_auc: 0.5024 - val_loss: 0.7048 - learning_rate: 0.0010
Epoch 2/300
811/811 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - accuracy: 0.4986 - auc: 0.4972 - loss: 0.7037
Epoch 2: val_loss improved from 0.70480 to 0.69420, saving model to models/best_model.keras
811/811 ━━━━━━━━━━━━━━━━━━━━ 67s 83ms/step - accuracy: 0.4986 - auc: 0.4972 - loss: 0.7037 - val_accuracy: 0.5007 - val_auc: 0.4984 - val_loss: 0.6942 - learning_rate: 0.0010
Epoch 3/300
811/811 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - accuracy: 0.5071 - auc: 0.5076 - loss: 0.6947
Epoch 3: val_loss did not improve from 0.69420
811/811 ━━━━━━━━━━━━━━━━━━━━ 70s 86ms/step - accuracy: 0.5071 - 

In [13]:
X_test, y_test = DataProcessor.make_sequences(df_test, columns_to_select=columns, window_size=48, forecast_horizon=1, classification=True)

In [14]:
lstm_forecaster.evaluate(X_test, y_test)

36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.4951 - auc: 0.4846 - loss: 0.6950
Test Loss: 0.6934801936149597, MSE: 0.5086805820465088, Directional Accuracy: <function directional_accuracy at 0x125a02840>
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step
Confidence-Filtered Accuracy: 0.50, Coverage: 100.00%


0.6934801936149597